# 主流视频编码压缩技术算法分析

 
## 一、MPEG-1技术介绍

1. MPEG-1标准发布于1992年，主要应用于VCD、MP3音乐等。
2. 使用MPEG-1的压缩算法可将一部120分钟的电影压缩到1.2GB左右大小，因此，它被广泛应用于VCD制作中。
3. MPEG-1采用块方式运动补偿、离散余弦变换、量化等技术。
4. 是Video CD的核心编码技术。
5. 在空间方向上

- 在图像压缩采用JPEG压缩算法去掉画面内部的冗余信息，即基于DCT的压缩技术，减少空间域冗余。
    
6. 在时间方向上

   采用基于16X16子块的运动补偿。
    
    
### 1、 MPEG-1的层次及语法结构


 ![](https://github.com/chensongpoixs/cvideo_codec/raw/master/%E7%AC%AC%E4%B8%89%E7%AB%A0%20%E8%A7%86%E9%A2%91%E7%BC%96%E7%A0%81%E5%8E%8B%E7%BC%A9%E6%8A%80%E6%9C%AF/img/data_mpeg_1.png)

1. MPEG-1的数据结构分为6层，从上到下依次是：
 
 
  ![](https://github.com/chensongpoixs/cvideo_codec/raw/master/%E7%AC%AC%E4%B8%89%E7%AB%A0%20%E8%A7%86%E9%A2%91%E7%BC%96%E7%A0%81%E5%8E%8B%E7%BC%A9%E6%8A%80%E6%9C%AF/img/data_mpeg_1_sgpsmb.png)
####  ①、运动补偿序列(Sequence)

    有表头+图片组+结束标志
 
#### ②、图片组（GOP）

    由一系列图片构成。
 
#### ③、 图片（Picture）

    是基本编码单元，包含一个亮度信号和两个色度信号。
 
#### ④、 图片切片（Slice）

    由一个或多个连续的宏块构成。

#### ⑤、 宏块（MB）

    运动补偿的基本单元。

 
 宏块 ---- 运动补偿的基本单元
 
 块 ----DCT操作的基本单元
 
 
 
### 2、 MPEG的图片组（GOP）

####  ①、为了在高效编码压缩的情况下， 获得可随机存储的高压缩比、高质量图像、MPEG定义了I、P、B三种帧类型。
   
   I： 帧内图（Intra Pictrue）
   P： 预测图（Predicted Picture）
   B： 双向图 (Bidirectional Pictrue)
   
I,P,B 表示1/25s时间间隔的帧序列画面。

MPEG的GOP结构图 （如下图所示）

![](https://github.com/chensongpoixs/cvideo_codec/raw/master/%E7%AC%AC%E4%B8%89%E7%AB%A0%20%E8%A7%86%E9%A2%91%E7%BC%96%E7%A0%81%E5%8E%8B%E7%BC%A9%E6%8A%80%E6%9C%AF/img/data_mpeg_gop.png)

#####  I帧（帧内图）

I帧采用类似JPEG的编码方式实现，它不以任何其他帧做参考，仅仅进行帧内的空域冗余压缩。

I帧的编码过程简单

 1. 图像进行彩色空间变换，从RGB到YCrCb
 2. 进行区块分割
 3. 对每个区块进行DCT变换
 4. 对DC系数进行差分脉冲编码
 5. 对AC系数进行Zig-Zig的形状排序，然后进行行程编码(RLE)
 6. 最后用Huffman编码。
 
 
数据量大

 由于I帧图像是不参考其他图像帧,只利用本帧的信息进行编码(即无运动预测,采用自身相关性), 因此数据量大。

随机存取

  由于图像序列间无相关性,因此可随机进入图像序列进行编码。
  
I帧编码原理

![](https://github.com/chensongpoixs/cvideo_codec/raw/master/%E7%AC%AC%E4%B8%89%E7%AB%A0%20%E8%A7%86%E9%A2%91%E7%BC%96%E7%A0%81%E5%8E%8B%E7%BC%A9%E6%8A%80%E6%9C%AF/img/data_mpeg_I_encode_frame.png)
 
##### P帧（预测帧）

 P帧是由一个过去的I帧或P帧采用运动补偿的帧间预测进行更有效编码的方法。
 
 预测图像P使用两种类型的参数来表示：

- 一种参数是当前要编码的图像宏块与参考图像的宏块之间的差值
- 一种是宏块的运动矢量
   
P帧的特点是其本身是前I帧或P帧的前向预测结果,也是产生下一个P帧的基准参考图像。

- 优点：可提供更大的压缩比
- 缺点：前一P帧是下一P帧补偿预测的基准, 如果前者存在误码, 则后者会将编码误差积累起来，传播下去。

P帧的压缩算法原理

![](https://github.com/chensongpoixs/cvideo_codec/raw/master/%E7%AC%AC%E4%B8%89%E7%AB%A0%20%E8%A7%86%E9%A2%91%E7%BC%96%E7%A0%81%E5%8E%8B%E7%BC%A9%E6%8A%80%E6%9C%AF/img/data_mpeg_frame_p_encode.png)

##### B帧（双向图帧）

- B帧空域提供最高的压缩比, 它即可以用过去的图像帧(I帧或者P帧),也可以用后来的图像(I帧或者P帧)进行运动补偿的双向预测编码方式。
- 由于B帧空域参考下一帧的信息进行编码, 从而减少B帧的大小, 相对P帧更小。
- B帧是同时以前面的I帧或P帧和后面的P帧或I帧为基准进行运动补偿所产生的图像,即双向预测编码。
- 以前面的I帧或P帧代表"过去的信息",  后面的P帧或I帧代表"未来的信息".
- 由于同时使用"过去" 和 "未来"信息, 故称为双向预测帧。

- B帧的压缩算法原理

![](https://github.com/chensongpoixs/cvideo_codec/raw/master/%E7%AC%AC%E4%B8%89%E7%AB%A0%20%E8%A7%86%E9%A2%91%E7%BC%96%E7%A0%81%E5%8E%8B%E7%BC%A9%E6%8A%80%E6%9C%AF/img/data_mpeg_frame_b_encode.png)


##### GOP 类型与尺寸

- GOP类型：是指GOP中, I, B, P帧的构成情况, 如, 是否含有B、P帧以及B、P帧的发布情况， 如：

   - 两个I帧之间有多少个B、P帧
   - I、p帧之间多少个B帧等
   
- GOP尺寸： 是指多少个帧之间会出现一个I帧。

   例如：

   - GOP结构： IBBPBBPBBPBBPBBPBBPBBPBBPBBI
   - GOP的尺寸：20， 即每20个帧出现一个I帧
   - GOP的类型：IBBP的结构
   

##### 传输与解码显示顺序

- 由于视频编码过程中，需要进行单向或双向参考预测, 因此，图像编码压缩传输及显示顺序并非一致。
- 在编码完成后, 图像不是以显示顺序传输的。因为参照图像I帧或P帧必须先于B帧图像恢复之前恢复,所以编码器需要对上述图像重新排序。也就是说在任何P帧或B帧被解码之前,必须有参考图像帧。
- MPEG编码传输及显示顺序

![](https://github.com/chensongpoixs/cvideo_codec/raw/master/%E7%AC%AC%E4%B8%89%E7%AB%A0%20%E8%A7%86%E9%A2%91%E7%BC%96%E7%A0%81%E5%8E%8B%E7%BC%A9%E6%8A%80%E6%9C%AF/img/data_mpeg_encode_show_I_B_P.png)

## 二、MPEG-2

- MPEG-2是DVD产品的核心编码技术
- MPEG-2包含两部分
   - 第一部分： MPEG-2系统描述部分
         - 用来在非可靠介质上传输数位视频信号和音频信号的机制
   - 第二部分： MPEG-2视频部分
         - 与MPEG-1类似,区别是它提供对隔行扫描视频模式的支持（隔行扫描广泛应用广播电视领域）
- MPEG-2 向后兼容
   - 即所有符合标准的MPEG-2编码器也能够正常部分MPEG-1视频流。
   
- MPEG-1与MPEG-2的区别

![](https://github.com/chensongpoixs/cvideo_codec/raw/master/%E7%AC%AC%E4%B8%89%E7%AB%A0%20%E8%A7%86%E9%A2%91%E7%BC%96%E7%A0%81%E5%8E%8B%E7%BC%A9%E6%8A%80%E6%9C%AF/img/data_mpeg_1_to_mepg_2.png)

- MPEG-2的编码步骤
  - 输入的图像首先进行颜色空间转换(RGB to YCrCb), 得到分量Y, Cr和Cb；
  - 将Y, Cr和Cb分量进行块分割,然后分别形成宏块,每个宏块子分割成8X8的小块
  - 对于I帧编码, 整个图像直接进入编码过程
  - 对于P帧和B帧, 首先需要做运动补偿
      - 由于相邻帧之间的相关性很强,各个宏块空域在前后帧中对应得位置知道相似的匹配块,该偏移量作为运动补偿向量被向量记录下来；
      - 运动估计重构区域的残差被送到编码器中进行编码；
         - 对残差(8X8块)进行DCT变换
         - 得到的系数进行量化和重新排序
  - 最后做行程和Huffman编码压缩
  
- MPEG-2的编码图形过程

  ![](https://github.com/chensongpoixs/cvideo_codec/raw/master/%E7%AC%AC%E4%B8%89%E7%AB%A0%20%E8%A7%86%E9%A2%91%E7%BC%96%E7%A0%81%E5%8E%8B%E7%BC%A9%E6%8A%80%E6%9C%AF/img/data_mpeg_encode_frame_huffman.png)
  

## 三、MPEG-4

###  算法核心：支持 "基于内容" 的编码与解码功能. 就是对场景中使用分割算法抽出的单独物理对象进行编码和解码。
### MPEG-4标准规定了各种音频视频对象的编码，除了包括自然的音频视频对象，还包含文字、2D和3D图形以及合成语言等等。
### MPEG-4通过描述场景结构信息,即各种对象的空间位置和时间关系等, 建立一个多媒体场景, 并将它与编码的对象一起传输。
###  由于对各个对象进行独立地编码, 从而空域达到很高的压缩比率，同时也为在接受端根据需要对内容进行差值提供了可能，适应了多媒体应用中的"人机交互"的需求。
- 分类
  - 合成视频编码
  - 自然视频编码
  
### MPEG-4技术详述
  - MPEG-4采用的仍然是类似于MPEG-4的基本编码框架，即典型的三步：预测编码、变换量化和熵编码；
  - 区别是基于优化思想进行设计,将先前标准中的某些技术加以改进。
     - 例如在原来的基础上提出${1/4}$和${1/8}$像素精度的运动补偿技术, 是的预测编码的性能大大提高
     - MPEG-4标准不仅仅给出具体的压缩算法，它是针对数字电视，交互式多媒体应用，视频监控等整合及压缩技术的需要而制定的。
  - MPEG标准中引入了视频对象的概念，打破了过去以 "宏块" 为单位编码的限制，其目的在于采用现代图像编码方法，利用人眼的视觉特性，抓住图像信息的本质，从轮廓、纹理的思路出发，支持基于视频内容交互功能。
  - "区别对待", 而非 "一视同仁"。 例如，在视频画面中，经常会遇到 "相同的场景, 不同的背景", 如果能够对场景进行分类并按照不同分类的特点有区别地进行编码压缩，将会给编码效率带来很大的提升空间。MPEG正是如此的"基于对象"的视频编码方式。
  
  - VO与VOP概念的引入
     - 传统的视频编码方式是将整个视频信号作为一个内容整体进行处理，其本身不可再分割，而这与人类对视觉信息的识别习惯是不同的。
     - MPEG-1和MPEG-2不能将一个视频信息完整地从视频信号中提取出来了。比如，将加有电视台台标和字幕的视频恢复成无台标、无字幕的视频。
     - 解决方法：
       - 编码时， 将不同的视频信息截体，即视频对象VO（Video Objects）区分对待。
       - 即分别独立的决心编码与传输，将图像序列中的每一帧，看成是由不同的VO加上活动背景所组成。
       - VO可以是人、车、动物等。
     - VOP（Video Object Plane）视频对象平面
     - 是视频对象VO在某一时刻的采样，VOP是MPEG-4视频编码的核心概念。
     - VOP的编码主要由两个部分
       - 一个形状编码
       - 一个是纹理和运动信息编码
       
  - MPEG-4是以VOP为单位进行编码和解码的。
  - MPEG-4的VO概念原理如图：
  
  ![](https://github.com/chensongpoixs/cvideo_codec/raw/master/%E7%AC%AC%E4%B8%89%E7%AB%A0%20%E8%A7%86%E9%A2%91%E7%BC%96%E7%A0%81%E5%8E%8B%E7%BC%A9%E6%8A%80%E6%9C%AF/img/data_mpeg_4_vop_vo.png)
  
 - VOP编码的类型
   - 内部VOP（I-VOP）:只用当前帧的信息编码
   - 单项预测VOP（P-VOP）：参考前面的I帧或P-VOP,利用运动补偿技术来编码。
   - 双向预测VOP（B-VOP）: 参考前后的I帧或P-VOP,利用运动补偿技术来编码。
   - 全景VOP（S-VOP）：用来编码Sprite对象。
      - Sprite编码技术
         - Sprite对象是针对背景对象的特点提出的。
     
 ![](https://github.com/chensongpoixs/cvideo_codec/raw/master/%E7%AC%AC%E4%B8%89%E7%AB%A0%20%E8%A7%86%E9%A2%91%E7%BC%96%E7%A0%81%E5%8E%8B%E7%BC%A9%E6%8A%80%E6%9C%AF/img/data_mpeg_4_sprite_backgroun.png)
   


![](https://github.com/chensongpoixs/cvideo_codec/raw/master/%E7%AC%AC%E4%B8%89%E7%AB%A0%20%E8%A7%86%E9%A2%91%E7%BC%96%E7%A0%81%E5%8E%8B%E7%BC%A9%E6%8A%80%E6%9C%AF/img/data_mpeg_vo_vop_encode.png)


       